In [ ]:
using POMDPs
using POMDPTools
using QuickPOMDPs
using MCTS

using Distributions
using LinearAlgebra

# Parameters

In [ ]:
# Dimensions
max_progress = 5
max_power = 20
max_h2o = 20
max_o2 = 20
max_food = 20

# Uncertainties
p_mission_regression = 0.05
p_power_fail = 0.02
p_water_loss = 0.02
p_food_loss = 0.02
p_no_mission_progress = 0.05

# Resource consumption
consumed_power = -10 # generated
consumed_h2o = 1
consumed_o2 = 8
consumed_food = 9

In [ ]:
# Define all possible states and actions (with constraints)
# [progress, power, h2o, o2, food]
indexed = LinearIndices((max_progress, max_power, max_h2o, max_o2, max_food))
states = 1:indexed[max_progress, max_power, max_h2o, max_o2, max_food]

basic_actions = [
      CartesianIndex(1, -1, 0, 0, 0), # make mission progress
      CartesianIndex(0, -1, 2, 0, 0), # make water
      CartesianIndex(0, -2, -1, 1, 0), # make oxygen
      CartesianIndex(0, -1, -1, -1, 2), # make food
      CartesianIndex(0, 0, 0, 0, 0)
] # do nothing
consumed = CartesianIndex(0, consumed_power, consumed_h2o, consumed_o2, consumed_food)
actions = [action - consumed for action in basic_actions]

initialindex = indexed[1, 20, 20, 20, 20]
initialstate = Deterministic(initialindex)

In [ ]:
# Determine if action is out of bounds
# Associated consequences are reflected in transition/reward functions
state_maxes = [max_progress, max_power, max_h2o, max_o2, max_food]
function outofbounds(state, action)
    for i in 1:length(state)
        if ((state[i] + action[i]) < 1 || (state[i] + action[i]) > state_maxes[i])
            return true
        end
    end
    return false
end

In [ ]:
# Transition function
function transition(s, a)
    svec = CartesianIndices((max_progress, max_power, max_h2o, max_o2, max_food))[s]
    svec = collect(Tuple(svec))
    a = collect(Tuple(a))
    if svec[2] + a[2] > max_power
        svec[2] = max_power
    end
    if outofbounds(svec, a)
        spvec = svec
    else
        x = rand()
        if (x < p_no_mission_progress)
            spvec = svec # no progress
        elseif (x < (p_water_loss + p_no_mission_progress))
            svec[3] = 1 # lose all water
            spvec = svec + a
        elseif (x<(p_water_loss + p_power_fail + p_no_mission_progress))
            svec[2] = 1 # lose all energy
            spvec = svec + a
        elseif (x < (p_mission_regression + p_water_loss + p_power_fail + p_no_mission_progress) && svec[3] > 1)
            svec[1] = svec[1] - 1 # mission setback
            spvec = svec + a
        else 
            spvec = svec + a
        end
    end
    spvec = CartesianIndex(Tuple(spvec))
    sp = indexed[spvec]
    return sp
end


In [ ]:
mission = QuickMDP(
    states = states,
    actions = actions,
    initialstate = initialstate,
    discount = 0.95,
    isterminal = s -> CartesianIndices((max_progress, max_power, max_h2o, max_o2, max_food))[s][1] == max_progress,

    transition = function(s,a)
        return Deterministic(transition(s,a))
    end,

    reward = function (s, a)
        svec = CartesianIndices((max_progress, max_power, max_h2o, max_o2, max_food))[s]
        spvec = svec + a
        if spvec[1] == max_progress
            r = 1000
        elseif outofbounds(svec, a) # If invalid action
            r = -1e9
        else
            r = -1
        end
        return r
    end
)

In [ ]:
solver = MCTSSolver(n_iterations=1000, depth=100, exploration_constant=5.0)
policy = solve(solver, mission)

In [ ]:
for i in 1:length(states)
    s = CartesianIndices((max_progress, max_power, max_h2o, max_o2, max_food))[i] 
    a = action(policy, i)
    if rand() < 0.1
        println(i)
        println(a + consumed)
    end
end